In [61]:
#Imports
from google.colab import drive
import pandas as pd
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense, LSTM, Flatten, Embedding, Dropout , Activation, GRU, Flatten, Input, Bidirectional, GlobalMaxPool1D, Convolution1D, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers

In [15]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
data = pd.read_csv('/content/drive/My Drive/Fixy/merged_de_da_final_X.csv')
print(data.shape)
data.head()

(3605229, 2)


,line,y
0,bunun x baş parmağı yok.,0
1,- bren x leigh.,1
2,ya şimdi buradan defolursunuz ya x karşınızda ...,0
3,öyle x diyebiliriz.,0
4,"kırmızıydı, ora x duruyor işte.",1


In [102]:
X = data['line']
y = data['y']

In [103]:
vocab_size = 15000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X)
list_tokenized_train = tokenizer.texts_to_sequences(X)

In [104]:
maxlen = 7
X_train_padded = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y, test_size=0.2, stratify=y)

In [106]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True, verbose=1)

In [113]:
embed_size = 128
model = Sequential()
model.add(Embedding(vocab_size, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, None, 128)         1920000   
_________________________________________________________________
bidirectional_31 (Bidirectio (None, None, 128)         98816     
_________________________________________________________________
bidirectional_32 (Bidirectio (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 65        
Total params: 2,060,097
Trainable params: 2,060,097
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])

batch_size = 10000
epochs = 7
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping_cb])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2595764 samples, validate on 288419 samples
Epoch 1/7
2595764/2595764 [==============================] - 29s 11us/step - loss: 0.3121 - auc: 0.8672 - accuracy: 0.8635 - val_loss: 0.2275 - val_auc: 0.9397 - val_accuracy: 0.9102
Epoch 2/7
2595764/2595764 [==============================] - 27s 10us/step - loss: 0.2211 - auc: 0.9494 - accuracy: 0.9134 - val_loss: 0.2175 - val_auc: 0.9549 - val_accuracy: 0.9146
Epoch 3/7
2595764/2595764 [==============================] - 27s 10us/step - loss: 0.2112 - auc: 0.9580 - accuracy: 0.9179 - val_loss: 0.2130 - val_auc: 0.9602 - val_accuracy: 0.9169
Epoch 4/7
2595764/2595764 [==============================] - 27s 10us/step - loss: 0.2044 - auc: 0.9619 - accuracy: 0.9211 - val_loss: 0.2105 - val_auc: 0.9632 - val_accuracy: 0.9186
Epoch 5/7
2595764/2595764 [==============================] - 27s 10us/step - loss: 0.1984 - auc: 0.9643 - accuracy: 0.9240 - val_loss: 0.2069 - val_auc: 0.9652 - val_accuracy: 0.9200
Epoch 6/7
2595764/2595764 [=====

In [138]:
pred = tokenizer.texts_to_sequences(["olsun demek x zor artık"])
print(pred)
maxlen = 7
padded_pred = pad_sequences(pred, maxlen=maxlen)
print(padded_pred)
model.predict(padded_pred)

#!!!!!#gerçekleri tarih yazar tarihi x galatasaray

#twitterda insanlar ne kadar x çok -de-da hatası yapıyor

#önemliler
#gömleğin önünü x iliklersen iyi olur
#yediğin de içtiğin x senin olsun bize gördüklerini anlat
#kimselere x bakmadım senden daha güzel

#olsun demek x zor artık
#sen x başını alıp gitme ne olur

#böyle koşullar x çalışmak çok sıkıcı 99
#senin gibi insanlar x çok çalışkan oluyor 0.05
###sen x geldiğinde gidelim
###sen de geldiğin x gidelim
###deden x gelsin 
###bu tişörtü x giymezsin abi ya
###elin x kini bırak
###sen x değilsin
#elin x kaldı
#elini x ayağını x çek
#dedendeki demiri x delsin
#böyle bir durum x sen olsan ne yapardın
#böyle bir durum x olsa haklı değilsin
#elin x ayağın da çok hareket ediyor
#ne x olsa işi alamadı
#YYY aklım da karışık zihnim x karışık
#kalbim ondan yana x olsa işi alamadı
#kalbim x acıyor 3Y
#bana klavye x lazım
#kendim x yapamadım

#olsa
#yabancı ülke de olsa yaşayamam
#küçük ülke de olsa çok güçlü
#küçük bir ülke de yaşamak istiyorum
#DENE##H  bu ülke x zengin bir ülke 

#ekler
#Bu yaz İstanbul' x yağmur yağdı
#Bu yaz İstanbul'a x yağmur yağdı

#ahmet x kalacağım
#ahmet x gelecek

#bu yaz bizimkiler x tatile gelecek
#bu yaz bizimkiler x tatil yapacağım

#20 sınıf x olsa yetmez

#bu yaz köye x yağmur yağdı
#bu yaz köy x yağmur yağdı

############annem dolap x yemek var dedi
############annem dolap x var dedi

#Siz x mi okul müdürüydünüz

#H##bu bir yarışma x olsa çirkefleşme

#HATALIDEDATAKINTISI
#H##bu durumlar x sakıncalı
#H##bu deneme x doğru buldun
#bu yaz köye x yağmur yağdı // yanlışını yaz
#seninle gelsem x gitsem de olmaz  bulamadı
#afrika x amerika x kötü durumda 
#sendeki şişe X bira mı var

[[208, 99, 1, 275, 92]]
[[  0   0 208  99   1 275  92]]


array([[0.04085088]], dtype=float32)

In [115]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

y_pred = model.predict_classes(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data: %.2f%%" % (accuracy * 100.0))

print(confusion_matrix(y_test, y_pred ))
print(classification_report(y_test, y_pred, digits=4))
print(roc_auc_score(y_test, y_pred))

Accuracy on Test Data: 92.13%
[[336706  20522]
 [ 36227 327591]]
              precision    recall  f1-score   support

           0     0.9029    0.9426    0.9223    357228
           1     0.9410    0.9004    0.9203    363818

    accuracy                         0.9213    721046
   macro avg     0.9220    0.9215    0.9213    721046
weighted avg     0.9221    0.9213    0.9213    721046

0.9214887915247114


In [116]:
model.save("Model_deda.h5")
print("Saved model to disk")

Saved model to disk


In [119]:
model.load_weights("/content/Model_deda.h5")

In [141]:
with open('tokenizer_deda.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)